In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly.express as px
import datetime
from statsmodels.tsa.deterministic import DeterministicProcess, CalendarFourier

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/store-sales-time-series-forecasting/oil.csv
/kaggle/input/store-sales-time-series-forecasting/sample_submission.csv
/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv
/kaggle/input/store-sales-time-series-forecasting/stores.csv
/kaggle/input/store-sales-time-series-forecasting/train.csv
/kaggle/input/store-sales-time-series-forecasting/test.csv
/kaggle/input/store-sales-time-series-forecasting/transactions.csv


In [2]:
oil = pd.read_csv('../input/store-sales-time-series-forecasting/oil.csv')
stores = pd.read_csv('../input/store-sales-time-series-forecasting/stores.csv')
holiday = pd.read_csv('../input/store-sales-time-series-forecasting/holidays_events.csv')
transaction = pd.read_csv('../input/store-sales-time-series-forecasting/transactions.csv')

train = pd.read_csv('../input/store-sales-time-series-forecasting/train.csv')
test = pd.read_csv('../input/store-sales-time-series-forecasting/test.csv')
submition = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/sample_submission.csv')

In [3]:
train

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0
1,1,2013-01-01,1,BABY CARE,0.000,0
2,2,2013-01-01,1,BEAUTY,0.000,0
3,3,2013-01-01,1,BEVERAGES,0.000,0
4,4,2013-01-01,1,BOOKS,0.000,0
...,...,...,...,...,...,...
3000883,3000883,2017-08-15,9,POULTRY,438.133,0
3000884,3000884,2017-08-15,9,PREPARED FOODS,154.553,1
3000885,3000885,2017-08-15,9,PRODUCE,2419.729,148
3000886,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8


In [4]:
test

,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2
3,3000891,2017-08-16,1,BEVERAGES,20
4,3000892,2017-08-16,1,BOOKS,0
...,...,...,...,...,...
28507,3029395,2017-08-31,9,POULTRY,1
28508,3029396,2017-08-31,9,PREPARED FOODS,0
28509,3029397,2017-08-31,9,PRODUCE,1
28510,3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,9


In [5]:
train['TRAINorTEST']=1
test['TRAINorTEST']=0

In [6]:
test

,id,date,store_nbr,family,onpromotion,TRAINorTEST
0,3000888,2017-08-16,1,AUTOMOTIVE,0,0
1,3000889,2017-08-16,1,BABY CARE,0,0
2,3000890,2017-08-16,1,BEAUTY,2,0
3,3000891,2017-08-16,1,BEVERAGES,20,0
4,3000892,2017-08-16,1,BOOKS,0,0
...,...,...,...,...,...,...
28507,3029395,2017-08-31,9,POULTRY,1,0
28508,3029396,2017-08-31,9,PREPARED FOODS,0,0
28509,3029397,2017-08-31,9,PRODUCE,1,0
28510,3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,9,0


In [7]:
train.dtypes

id               int64
date            object
store_nbr        int64
family          object
sales          float64
onpromotion      int64
TRAINorTEST      int64
dtype: object

In [8]:
train

,id,date,store_nbr,family,sales,onpromotion,TRAINorTEST
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0,1
1,1,2013-01-01,1,BABY CARE,0.000,0,1
2,2,2013-01-01,1,BEAUTY,0.000,0,1
3,3,2013-01-01,1,BEVERAGES,0.000,0,1
4,4,2013-01-01,1,BOOKS,0.000,0,1
...,...,...,...,...,...,...,...
3000883,3000883,2017-08-15,9,POULTRY,438.133,0,1
3000884,3000884,2017-08-15,9,PREPARED FOODS,154.553,1,1
3000885,3000885,2017-08-15,9,PRODUCE,2419.729,148,1
3000886,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,1


In [9]:
oil

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20
...,...,...
1213,2017-08-25,47.65
1214,2017-08-28,46.40
1215,2017-08-29,46.46
1216,2017-08-30,45.96


In [10]:
transaction

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922
...,...,...,...
83483,2017-08-15,50,2804
83484,2017-08-15,51,1573
83485,2017-08-15,52,2255
83486,2017-08-15,53,932


In [11]:
stores.head(2)

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13


In [12]:
holiday.head(2)

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False


In [13]:
holiday.shape

(350, 6)

In [14]:
holiday.locale_name.value_counts()

Ecuador                           174
Quito                              13
Riobamba                           12
Guaranda                           12
Latacunga                          12
Ambato                             12
Guayaquil                          11
Cuenca                              7
Ibarra                              7
Salinas                             6
Loja                                6
Santa Elena                         6
Santo Domingo de los Tsachilas      6
Quevedo                             6
Manta                               6
Esmeraldas                          6
Cotopaxi                            6
El Carmen                           6
Santo Domingo                       6
Machala                             6
Imbabura                            6
Puyo                                6
Libertad                            6
Cayambe                             6
Name: locale_name, dtype: int64

In [15]:
train.iloc[[0, -1]]

,id,date,store_nbr,family,sales,onpromotion,TRAINorTEST
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,1
3000887,3000887,2017-08-15,9,SEAFOOD,16.0,0,1


In [16]:
oil.iloc[[0,-1]]

,date,dcoilwtico
0,2013-01-01,NaN
1217,2017-08-31,47.26


In [17]:
transaction.iloc[[0,-1]]

,date,store_nbr,transactions
0,2013-01-01,25,770
83487,2017-08-15,54,802


In [18]:
stores.head(2)

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13


# Store +transaction

In [19]:
stores.shape

(54, 5)

In [20]:
transaction.shape

(83488, 3)

In [21]:
stores_trans=transaction.merge(stores,on="store_nbr",how="inner")


In [22]:
stores_trans

,date,store_nbr,transactions,city,state,type,cluster
0,2013-01-01,25,770,Salinas,Santa Elena,D,1
1,2013-01-02,25,1038,Salinas,Santa Elena,D,1
2,2013-01-03,25,887,Salinas,Santa Elena,D,1
3,2013-01-04,25,1054,Salinas,Santa Elena,D,1
4,2013-01-05,25,1355,Salinas,Santa Elena,D,1
...,...,...,...,...,...,...,...
83483,2017-08-11,52,2957,Manta,Manabi,A,11
83484,2017-08-12,52,2804,Manta,Manabi,A,11
83485,2017-08-13,52,2433,Manta,Manabi,A,11
83486,2017-08-14,52,2074,Manta,Manabi,A,11


In [23]:
stores_trans.dtypes

date            object
store_nbr        int64
transactions     int64
city            object
state           object
type            object
cluster          int64
dtype: object

In [24]:
print("the range of date")
stores_trans.date.iloc[[0, -1]]

the range of date


0        2013-01-01
83487    2017-08-15
Name: date, dtype: object

In [25]:
print("the number of stores")

stores_trans.groupby('store_nbr').store_nbr.nunique().count()

the number of stores


54

In [26]:
print("the number of cities")

stores_trans.groupby('city').city.nunique().count()

the number of cities


22

In [27]:
print("the number of states")

stores_trans.groupby('state').state.nunique().count()

the number of states


16

In [28]:
print("the number of types")

stores_trans.groupby('type').type.nunique().count()

the number of types


5

In [29]:
print("the number of cluster")

stores_trans.groupby('cluster').cluster.nunique().count()

the number of cluster


17

In [30]:
stores_trans.date=pd.to_datetime(stores_trans['date'], format='%Y-%m-%d ')


In [31]:
# stores_trans.groupby(["store_nbr"]).agg({'transactions' : np.sum }).reset_index().sort_values(by=['transactions'], ascending=False)

In [32]:
temp=stores_trans.groupby(["store_nbr","type"]).agg({'transactions' : np.sum }).reset_index()


In [33]:
temp.store_nbr=temp.store_nbr.astype(str)

**Number of transacions by cluster type**

In [34]:
px.bar(temp.sort_values(by=['transactions'], ascending=False),x="store_nbr",y="transactions",color="type")

**Number of transacions by cluster**

In [35]:
temp2=stores_trans.groupby(["store_nbr","type","cluster"]).agg({'transactions' : np.sum }).reset_index()

In [36]:
temp2.cluster=temp2.cluster.astype(str)
temp2.store_nbr=temp2.store_nbr.astype(str)

In [37]:
px.bar(temp2.sort_values(by=['transactions'], ascending=False),x="store_nbr",y="transactions",color="cluster")

**Number of transacions by city**


In [38]:
temp3=stores_trans.groupby(["store_nbr","type","city"]).agg({'transactions' : np.sum }).reset_index()

In [39]:
temp3.store_nbr=temp2.store_nbr.astype(str)

In [40]:
px.bar(temp3.sort_values(by=['transactions'], ascending=False),x="store_nbr",y="transactions",color="city")

# Oil

In [41]:
oil.date=pd.to_datetime(oil['date'], format='%Y-%m-%d ')

In [42]:
px.scatter(oil,x='date',y='dcoilwtico')

In [43]:
temp4=stores_trans.groupby(["date"]).agg({'transactions' : np.sum }).reset_index()
px.scatter(temp4,x='date',y='transactions')

In [44]:
temp4

,date,transactions
0,2013-01-01,770
1,2013-01-02,93215
2,2013-01-03,78504
3,2013-01-04,78494
4,2013-01-05,93573
...,...,...
1677,2017-08-11,89551
1678,2017-08-12,89927
1679,2017-08-13,85993
1680,2017-08-14,85448


In [45]:
holiday.iloc[[0,-1]]

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
349,2017-12-26,Additional,National,Ecuador,Navidad+1,False


In [46]:
holiday.date=pd.to_datetime(holiday['date'],format='%Y-%m-%d')



In [47]:
trans_store_holiday=temp4.merge(holiday,on="date",how="left")


In [48]:
trans_store_holiday

,date,transactions,type,locale,locale_name,description,transferred
0,2013-01-01,770,Holiday,National,Ecuador,Primer dia del ano,False
1,2013-01-02,93215,NaN,NaN,NaN,NaN,NaN
2,2013-01-03,78504,NaN,NaN,NaN,NaN,NaN
3,2013-01-04,78494,NaN,NaN,NaN,NaN,NaN
4,2013-01-05,93573,Work Day,National,Ecuador,Recupero puente Navidad,False
...,...,...,...,...,...,...,...
1707,2017-08-11,89551,Transfer,National,Ecuador,Traslado Primer Grito de Independencia,False
1708,2017-08-12,89927,NaN,NaN,NaN,NaN,NaN
1709,2017-08-13,85993,NaN,NaN,NaN,NaN,NaN
1710,2017-08-14,85448,NaN,NaN,NaN,NaN,NaN


In [49]:
trans_store_holiday.duplicated().any().sum()

0

In [50]:
trans_store_holiday.isna().sum()

date               0
transactions       0
type            1431
locale          1431
locale_name     1431
description     1431
transferred     1431
dtype: int64

In [51]:
trans_store_holiday.dtypes

date            datetime64[ns]
transactions             int64
type                    object
locale                  object
locale_name             object
description             object
transferred             object
dtype: object

In [52]:
trans_store_holiday=trans_store_holiday.rename(columns={"type": "holiday"})
trans_store_holiday.holiday=trans_store_holiday.holiday.astype(str)

In [53]:
px.scatter(trans_store_holiday,x='date',y='transactions',color='holiday')

In [54]:
trans_store_holiday

,date,transactions,holiday,locale,locale_name,description,transferred
0,2013-01-01,770,Holiday,National,Ecuador,Primer dia del ano,False
1,2013-01-02,93215,nan,NaN,NaN,NaN,NaN
2,2013-01-03,78504,nan,NaN,NaN,NaN,NaN
3,2013-01-04,78494,nan,NaN,NaN,NaN,NaN
4,2013-01-05,93573,Work Day,National,Ecuador,Recupero puente Navidad,False
...,...,...,...,...,...,...,...
1707,2017-08-11,89551,Transfer,National,Ecuador,Traslado Primer Grito de Independencia,False
1708,2017-08-12,89927,nan,NaN,NaN,NaN,NaN
1709,2017-08-13,85993,nan,NaN,NaN,NaN,NaN
1710,2017-08-14,85448,nan,NaN,NaN,NaN,NaN


In [55]:
# px.scatter(trans_store_holiday.query('date.dt.year==2013'),x='date',y='transactions_all',color='holiday')

In [56]:
stores_trans.iloc[[0,-1]]

,date,store_nbr,transactions,city,state,type,cluster
0,2013-01-01,25,770,Salinas,Santa Elena,D,1
83487,2017-08-15,52,2255,Manta,Manabi,A,11


In [57]:
trans_store_holiday=trans_store_holiday.rename(columns={"transactions": "transactions_all"})


In [58]:
trans_store_holiday.iloc[[0,-1]]

,date,transactions_all,holiday,locale,locale_name,description,transferred
0,2013-01-01,770,Holiday,National,Ecuador,Primer dia del ano,False
1711,2017-08-15,86561,Holiday,Local,Riobamba,Fundacion de Riobamba,False


In [59]:
trans_store_holiday2=stores_trans.merge(trans_store_holiday,on="date",how="left")


In [60]:
trans_store_holiday2['transaction_perc']=trans_store_holiday2['transactions']/trans_store_holiday2['transactions_all']

In [61]:
trans_store_holiday2

,date,store_nbr,transactions,city,state,type,cluster,transactions_all,holiday,locale,locale_name,description,transferred,transaction_perc
0,2013-01-01,25,770,Salinas,Santa Elena,D,1,770,Holiday,National,Ecuador,Primer dia del ano,False,1.000000
1,2013-01-02,25,1038,Salinas,Santa Elena,D,1,93215,nan,NaN,NaN,NaN,NaN,0.011136
2,2013-01-03,25,887,Salinas,Santa Elena,D,1,78504,nan,NaN,NaN,NaN,NaN,0.011299
3,2013-01-04,25,1054,Salinas,Santa Elena,D,1,78494,nan,NaN,NaN,NaN,NaN,0.013428
4,2013-01-05,25,1355,Salinas,Santa Elena,D,1,93573,Work Day,National,Ecuador,Recupero puente Navidad,False,0.014481
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85002,2017-08-11,52,2957,Manta,Manabi,A,11,89551,Transfer,National,Ecuador,Traslado Primer Grito de Independencia,False,0.033020
85003,2017-08-12,52,2804,Manta,Manabi,A,11,89927,nan,NaN,NaN,NaN,NaN,0.031181
85004,2017-08-13,52,2433,Manta,Manabi,A,11,85993,nan,NaN,NaN,NaN,NaN,0.028293
85005,2017-08-14,52,2074,Manta,Manabi,A,11,85448,nan,NaN,NaN,NaN,NaN,0.024272


In [62]:
temp6=trans_store_holiday2.groupby(["date","city"]).agg({"transaction_perc" :np.sum}).reset_index()

In [63]:
# temp6.query('date=='2013-01-02'')

temp6[temp6['date'] == '2013-01-02']

,date,city,transaction_perc
1,2013-01-02,Ambato,0.047825
2,2013-01-02,Babahoyo,0.015030
3,2013-01-02,Cayambe,0.038052
4,2013-01-02,Cuenca,0.035166
5,2013-01-02,Daule,0.014869
6,2013-01-02,El Carmen,0.010706
7,2013-01-02,Esmeraldas,0.014858
8,2013-01-02,Guaranda,0.014686
9,2013-01-02,Guayaquil,0.115389
10,2013-01-02,Ibarra,0.017401


In [64]:
px.pie(temp6[temp6['date'] == '201-01-02'], values='transaction_perc', names='city', title='percentage of transactions in each city')

In [65]:
px.pie(temp6[temp6['date'] == '201-01-02'], values='transaction_perc', names='city', title='percentage of transactions in each city')

# train +test prepare

In [66]:
train.date=pd.to_datetime(train['date'],format="%Y-%m-%d")
test.date=pd.to_datetime(test['date'],format="%Y-%m-%d")



In [67]:
train.dtypes

id                      int64
date           datetime64[ns]
store_nbr               int64
family                 object
sales                 float64
onpromotion             int64
TRAINorTEST             int64
dtype: object

In [68]:
train

,id,date,store_nbr,family,sales,onpromotion,TRAINorTEST
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0,1
1,1,2013-01-01,1,BABY CARE,0.000,0,1
2,2,2013-01-01,1,BEAUTY,0.000,0,1
3,3,2013-01-01,1,BEVERAGES,0.000,0,1
4,4,2013-01-01,1,BOOKS,0.000,0,1
...,...,...,...,...,...,...,...
3000883,3000883,2017-08-15,9,POULTRY,438.133,0,1
3000884,3000884,2017-08-15,9,PREPARED FOODS,154.553,1,1
3000885,3000885,2017-08-15,9,PRODUCE,2419.729,148,1
3000886,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,1


In [69]:
test

,id,date,store_nbr,family,onpromotion,TRAINorTEST
0,3000888,2017-08-16,1,AUTOMOTIVE,0,0
1,3000889,2017-08-16,1,BABY CARE,0,0
2,3000890,2017-08-16,1,BEAUTY,2,0
3,3000891,2017-08-16,1,BEVERAGES,20,0
4,3000892,2017-08-16,1,BOOKS,0,0
...,...,...,...,...,...,...
28507,3029395,2017-08-31,9,POULTRY,1,0
28508,3029396,2017-08-31,9,PREPARED FOODS,0,0
28509,3029397,2017-08-31,9,PRODUCE,1,0
28510,3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,9,0


In [70]:
train_test=pd.concat([train, test], axis=0)


In [71]:
train_test

,id,date,store_nbr,family,sales,onpromotion,TRAINorTEST
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,1
1,1,2013-01-01,1,BABY CARE,0.0,0,1
2,2,2013-01-01,1,BEAUTY,0.0,0,1
3,3,2013-01-01,1,BEVERAGES,0.0,0,1
4,4,2013-01-01,1,BOOKS,0.0,0,1
...,...,...,...,...,...,...,...
28507,3029395,2017-08-31,9,POULTRY,NaN,1,0
28508,3029396,2017-08-31,9,PREPARED FOODS,NaN,0,0
28509,3029397,2017-08-31,9,PRODUCE,NaN,1,0
28510,3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,NaN,9,0


In [72]:
trans_store_holiday2

,date,store_nbr,transactions,city,state,type,cluster,transactions_all,holiday,locale,locale_name,description,transferred,transaction_perc
0,2013-01-01,25,770,Salinas,Santa Elena,D,1,770,Holiday,National,Ecuador,Primer dia del ano,False,1.000000
1,2013-01-02,25,1038,Salinas,Santa Elena,D,1,93215,nan,NaN,NaN,NaN,NaN,0.011136
2,2013-01-03,25,887,Salinas,Santa Elena,D,1,78504,nan,NaN,NaN,NaN,NaN,0.011299
3,2013-01-04,25,1054,Salinas,Santa Elena,D,1,78494,nan,NaN,NaN,NaN,NaN,0.013428
4,2013-01-05,25,1355,Salinas,Santa Elena,D,1,93573,Work Day,National,Ecuador,Recupero puente Navidad,False,0.014481
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85002,2017-08-11,52,2957,Manta,Manabi,A,11,89551,Transfer,National,Ecuador,Traslado Primer Grito de Independencia,False,0.033020
85003,2017-08-12,52,2804,Manta,Manabi,A,11,89927,nan,NaN,NaN,NaN,NaN,0.031181
85004,2017-08-13,52,2433,Manta,Manabi,A,11,85993,nan,NaN,NaN,NaN,NaN,0.028293
85005,2017-08-14,52,2074,Manta,Manabi,A,11,85448,nan,NaN,NaN,NaN,NaN,0.024272


In [73]:
trans_store_holiday2.dtypes

date                datetime64[ns]
store_nbr                    int64
transactions                 int64
city                        object
state                       object
type                        object
cluster                      int64
transactions_all             int64
holiday                     object
locale                      object
locale_name                 object
description                 object
transferred                 object
transaction_perc           float64
dtype: object

In [74]:
train_test.dtypes

id                      int64
date           datetime64[ns]
store_nbr               int64
family                 object
sales                 float64
onpromotion             int64
TRAINorTEST             int64
dtype: object

# all

In [75]:
all=train_test.merge(trans_store_holiday2,on=["date","store_nbr"],how="left")

In [76]:
all.shape

(3079527, 19)

In [77]:
all.iloc[[0,-1]]

,id,date,store_nbr,family,sales,onpromotion,TRAINorTEST,transactions,city,state,type,cluster,transactions_all,holiday,locale,locale_name,description,transferred,transaction_perc
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3079526,3029399,2017-08-31,9,SEAFOOD,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [78]:
all[all.transactions.notnull()]

,id,date,store_nbr,family,sales,onpromotion,TRAINorTEST,transactions,city,state,type,cluster,transactions_all,holiday,locale,locale_name,description,transferred,transaction_perc
561,561,2013-01-01,25,AUTOMOTIVE,0.000,0,1,770.0,Salinas,Santa Elena,D,1.0,770.0,Holiday,National,Ecuador,Primer dia del ano,False,1.000000
562,562,2013-01-01,25,BABY CARE,0.000,0,1,770.0,Salinas,Santa Elena,D,1.0,770.0,Holiday,National,Ecuador,Primer dia del ano,False,1.000000
563,563,2013-01-01,25,BEAUTY,2.000,0,1,770.0,Salinas,Santa Elena,D,1.0,770.0,Holiday,National,Ecuador,Primer dia del ano,False,1.000000
564,564,2013-01-01,25,BEVERAGES,810.000,0,1,770.0,Salinas,Santa Elena,D,1.0,770.0,Holiday,National,Ecuador,Primer dia del ano,False,1.000000
565,565,2013-01-01,25,BOOKS,0.000,0,1,770.0,Salinas,Santa Elena,D,1.0,770.0,Holiday,National,Ecuador,Primer dia del ano,False,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3051010,3000883,2017-08-15,9,POULTRY,438.133,0,1,2155.0,Quito,Pichincha,B,6.0,86561.0,Holiday,Local,Riobamba,Fundacion de Riobamba,False,0.024896
3051011,3000884,2017-08-15,9,PREPARED FOODS,154.553,1,1,2155.0,Quito,Pichincha,B,6.0,86561.0,Holiday,Local,Riobamba,Fundacion de Riobamba,False,0.024896
3051012,3000885,2017-08-15,9,PRODUCE,2419.729,148,1,2155.0,Quito,Pichincha,B,6.0,86561.0,Holiday,Local,Riobamba,Fundacion de Riobamba,False,0.024896
3051013,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,1,2155.0,Quito,Pichincha,B,6.0,86561.0,Holiday,Local,Riobamba,Fundacion de Riobamba,False,0.024896


In [79]:
test.store_nbr.value_counts().shape

(54,)

In [80]:
all.family.value_counts().shape

(33,)

In [81]:
test.shape

(28512, 6)

We need to forecast the sales of each **family** for each **store** in each **day** and that for 15 days

In [82]:
# px.scatter(all[all.date.dt.year==2013],x='date',y='sales',color='family',facet_col='store_nbr', facet_col_wrap = 5,
#           facet_row_spacing = 0.03)

# Store

In [83]:
stores.head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [84]:
print('Not unique stores means the cities that contains more than 1 store')
stores.city.value_counts().reset_index().query('city>1')

Not unique stores means the cities that contains more than 1 store


,index,city
0,Quito,18
1,Guayaquil,8
2,Cuenca,3
3,Santo Domingo,3
4,Manta,2
5,Latacunga,2
6,Machala,2
7,Ambato,2


In [85]:
stores_trans

,date,store_nbr,transactions,city,state,type,cluster
0,2013-01-01,25,770,Salinas,Santa Elena,D,1
1,2013-01-02,25,1038,Salinas,Santa Elena,D,1
2,2013-01-03,25,887,Salinas,Santa Elena,D,1
3,2013-01-04,25,1054,Salinas,Santa Elena,D,1
4,2013-01-05,25,1355,Salinas,Santa Elena,D,1
...,...,...,...,...,...,...,...
83483,2017-08-11,52,2957,Manta,Manabi,A,11
83484,2017-08-12,52,2804,Manta,Manabi,A,11
83485,2017-08-13,52,2433,Manta,Manabi,A,11
83486,2017-08-14,52,2074,Manta,Manabi,A,11


In [86]:
def numstores_eachyear(df):
    for i in df.date.dt.year.unique():
        print('the number of stores in', i,' is :',df[df.date.dt.year==i].store_nbr.unique().shape[0])
    

In [87]:
numstores_eachyear(stores_trans)

the number of stores in 2013  is : 47
the number of stores in 2014  is : 48
the number of stores in 2015  is : 53
the number of stores in 2016  is : 53
the number of stores in 2017  is : 54


In [88]:
list=[19, 20, 21, 28, 35, 41, 51, 52]

for i in list:
    display(stores_trans[stores_trans.store_nbr==i])




,date,store_nbr,transactions,city,state,type,cluster
31574,2013-01-02,19,1369,Guaranda,Bolivar,C,15
31575,2013-01-03,19,1098,Guaranda,Bolivar,C,15
31576,2013-01-04,19,1003,Guaranda,Bolivar,C,15
31577,2013-01-05,19,1713,Guaranda,Bolivar,C,15
31578,2013-01-06,19,1157,Guaranda,Bolivar,C,15
...,...,...,...,...,...,...,...
33245,2017-08-11,19,1158,Guaranda,Bolivar,C,15
33246,2017-08-12,19,1578,Guaranda,Bolivar,C,15
33247,2017-08-13,19,931,Guaranda,Bolivar,C,15
33248,2017-08-14,19,1173,Guaranda,Bolivar,C,15


,date,store_nbr,transactions,city,state,type,cluster
79448,2015-02-13,20,4500,Quito,Pichincha,B,6
79449,2015-02-14,20,3698,Quito,Pichincha,B,6
79450,2015-02-15,20,3181,Quito,Pichincha,B,6
79451,2015-02-16,20,2915,Quito,Pichincha,B,6
79452,2015-02-17,20,2826,Quito,Pichincha,B,6
...,...,...,...,...,...,...,...
80352,2017-08-11,20,1751,Quito,Pichincha,B,6
80353,2017-08-12,20,1664,Quito,Pichincha,B,6
80354,2017-08-13,20,1834,Quito,Pichincha,B,6
80355,2017-08-14,20,1522,Quito,Pichincha,B,6


,date,store_nbr,transactions,city,state,type,cluster
81231,2015-07-24,21,3220,Santo Domingo,Santo Domingo de los Tsachilas,B,6
81232,2015-07-25,21,3109,Santo Domingo,Santo Domingo de los Tsachilas,B,6
81233,2015-07-26,21,3354,Santo Domingo,Santo Domingo de los Tsachilas,B,6
81234,2015-07-27,21,2143,Santo Domingo,Santo Domingo de los Tsachilas,B,6
81235,2015-07-28,21,2045,Santo Domingo,Santo Domingo de los Tsachilas,B,6
...,...,...,...,...,...,...,...
81974,2017-08-11,21,1165,Santo Domingo,Santo Domingo de los Tsachilas,B,6
81975,2017-08-12,21,1119,Santo Domingo,Santo Domingo de los Tsachilas,B,6
81976,2017-08-13,21,1403,Santo Domingo,Santo Domingo de los Tsachilas,B,6
81977,2017-08-14,21,1045,Santo Domingo,Santo Domingo de los Tsachilas,B,6


,date,store_nbr,transactions,city,state,type,cluster
39859,2013-01-02,28,950,Guayaquil,Guayas,E,10
39860,2013-01-03,28,749,Guayaquil,Guayas,E,10
39861,2013-01-04,28,651,Guayaquil,Guayas,E,10
39862,2013-01-05,28,1033,Guayaquil,Guayas,E,10
39863,2013-01-06,28,1207,Guayaquil,Guayas,E,10
...,...,...,...,...,...,...,...
41531,2017-08-11,28,1390,Guayaquil,Guayas,E,10
41532,2017-08-12,28,1284,Guayaquil,Guayas,E,10
41533,2017-08-13,28,1504,Guayaquil,Guayas,E,10
41534,2017-08-14,28,1064,Guayaquil,Guayas,E,10


,date,store_nbr,transactions,city,state,type,cluster
49902,2013-01-02,35,786,Playas,Guayas,C,3
49903,2013-01-03,35,725,Playas,Guayas,C,3
49904,2013-01-04,35,630,Playas,Guayas,C,3
49905,2013-01-05,35,723,Playas,Guayas,C,3
49906,2013-01-06,35,651,Playas,Guayas,C,3
...,...,...,...,...,...,...,...
51573,2017-08-11,35,847,Playas,Guayas,C,3
51574,2017-08-12,35,842,Playas,Guayas,C,3
51575,2017-08-13,35,658,Playas,Guayas,C,3
51576,2017-08-14,35,594,Playas,Guayas,C,3


,date,store_nbr,transactions,city,state,type,cluster
58289,2013-01-02,41,915,Machala,El Oro,D,4
58290,2013-01-03,41,809,Machala,El Oro,D,4
58291,2013-01-04,41,835,Machala,El Oro,D,4
58292,2013-01-05,41,1021,Machala,El Oro,D,4
58293,2013-01-06,41,1142,Machala,El Oro,D,4
...,...,...,...,...,...,...,...
59961,2017-08-11,41,1316,Machala,El Oro,D,4
59962,2017-08-12,41,1254,Machala,El Oro,D,4
59963,2017-08-13,41,1346,Machala,El Oro,D,4
59964,2017-08-14,41,1045,Machala,El Oro,D,4


,date,store_nbr,transactions,city,state,type,cluster
73377,2013-01-02,51,1985,Guayaquil,Guayas,A,17
73378,2013-01-03,51,1644,Guayaquil,Guayas,A,17
73379,2013-01-04,51,1786,Guayaquil,Guayas,A,17
73380,2013-01-05,51,2068,Guayaquil,Guayas,A,17
73381,2013-01-06,51,1934,Guayaquil,Guayas,A,17
...,...,...,...,...,...,...,...
75049,2017-08-11,51,1922,Guayaquil,Guayas,A,17
75050,2017-08-12,51,1590,Guayaquil,Guayas,A,17
75051,2017-08-13,51,1816,Guayaquil,Guayas,A,17
75052,2017-08-14,51,1446,Guayaquil,Guayas,A,17


,date,store_nbr,transactions,city,state,type,cluster
83370,2017-04-20,52,4068,Manta,Manabi,A,11
83371,2017-04-21,52,3151,Manta,Manabi,A,11
83372,2017-04-22,52,4046,Manta,Manabi,A,11
83373,2017-04-23,52,4209,Manta,Manabi,A,11
83374,2017-04-24,52,2682,Manta,Manabi,A,11
...,...,...,...,...,...,...,...
83483,2017-08-11,52,2957,Manta,Manabi,A,11
83484,2017-08-12,52,2804,Manta,Manabi,A,11
83485,2017-08-13,52,2433,Manta,Manabi,A,11
83486,2017-08-14,52,2074,Manta,Manabi,A,11


In [89]:

deb=stores_trans[stores_trans.date=="2013-01-02"]


In [90]:
stores_trans[~stores_trans.store_nbr.isin(deb.store_nbr)].store_nbr.unique()


array([36, 53, 20, 29, 21, 42, 22, 52])

In [91]:
stores_trans[stores_trans.date>"2013-01-02"].store_nbr.unique().shape


(54,)

In [92]:
stores_trans[stores_trans.date<="2013-01-02"].store_nbr.unique().shape


(46,)

In [93]:
def store_feat (data):
    
    df = data.copy()
    
    # Adding features to data
    df['unique_store'] = df.city.apply(lambda x: 0 if x in ['Quito', 'Guayaquil', 'Santo Domingo', 'Cuenca', 'Manta', 'Machala', 'Latacunga', 'Ambato'] else 1)
    df['new_store'] = df.store_nbr.apply(lambda x: 1 if x in [36, 53, 20, 29, 21, 42, 22, 52] else 0)
        
    # Merging orig_stores, orig_test and orig_train
    df = pd.concat([train, test], axis=0).merge(df, on=['store_nbr'], how='left')
    df = df.rename(columns={'type' : 'store'}) 

    return df

In [94]:
res_df=store_feat(stores)

# Events

In [95]:
holiday

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False
...,...,...,...,...,...,...
345,2017-12-22,Additional,National,Ecuador,Navidad-3,False
346,2017-12-23,Additional,National,Ecuador,Navidad-2,False
347,2017-12-24,Additional,National,Ecuador,Navidad-1,False
348,2017-12-25,Holiday,National,Ecuador,Navidad,False


**let just Non-transferred events**

In [96]:
holiday.loc[297, 'transferred']

True

In [97]:
holiday.query("transferred!=False")

,date,type,locale,locale_name,description,transferred
19,2012-10-09,Holiday,National,Ecuador,Independencia de Guayaquil,True
72,2013-10-09,Holiday,National,Ecuador,Independencia de Guayaquil,True
135,2014-10-09,Holiday,National,Ecuador,Independencia de Guayaquil,True
255,2016-05-24,Holiday,National,Ecuador,Batalla de Pichincha,True
266,2016-07-25,Holiday,Local,Guayaquil,Fundacion de Guayaquil,True
268,2016-08-10,Holiday,National,Ecuador,Primer Grito de Independencia,True
297,2017-01-01,Holiday,National,Ecuador,Primer dia del ano,True
303,2017-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,True
312,2017-05-24,Holiday,National,Ecuador,Batalla de Pichincha,True
324,2017-08-10,Holiday,National,Ecuador,Primer Grito de Independencia,True


In [98]:
 holiday.loc[297, 'transferred']  = False

In [99]:
df=holiday.query("transferred!=True")

In [100]:
#remove duplicates
df_holiday = df.drop(index=holiday[holiday[['date', 'locale_name']].duplicated()].index.values)

In [101]:
df_holiday

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False
...,...,...,...,...,...,...
345,2017-12-22,Additional,National,Ecuador,Navidad-3,False
346,2017-12-23,Additional,National,Ecuador,Navidad-2,False
347,2017-12-24,Additional,National,Ecuador,Navidad-1,False
348,2017-12-25,Holiday,National,Ecuador,Navidad,False


In [102]:
df_holiday.loc[df_holiday.type=='Event', 'type'] = df_holiday.description.apply(lambda x: x[0:7])


In [103]:
df_holiday.type.value_counts()

Holiday       210
Additional     48
Terremo        29
Mundial        13
Transfer       11
Bridge          5
Work Day        5
Dia de          4
Black F         3
Cyber M         3
Inaugur         1
Name: type, dtype: int64

In [104]:
df_holiday.locale.value_counts()

National    161
Local       147
Regional     24
Name: locale, dtype: int64

In [105]:
nat_df = df_holiday.query("locale=='National'")
loc_df = df_holiday.query("locale=='Local'")
reg_df = df_holiday.query("locale=='Regional'")

In [106]:
nat_df

,date,type,locale,locale_name,description,transferred
14,2012-08-10,Holiday,National,Ecuador,Primer Grito de Independencia,False
20,2012-10-12,Transfer,National,Ecuador,Traslado Independencia de Guayaquil,False
21,2012-11-02,Holiday,National,Ecuador,Dia de Difuntos,False
22,2012-11-03,Holiday,National,Ecuador,Independencia de Cuenca,False
31,2012-12-21,Additional,National,Ecuador,Navidad-4,False
...,...,...,...,...,...,...
345,2017-12-22,Additional,National,Ecuador,Navidad-3,False
346,2017-12-23,Additional,National,Ecuador,Navidad-2,False
347,2017-12-24,Additional,National,Ecuador,Navidad-1,False
348,2017-12-25,Holiday,National,Ecuador,Navidad,False


In [107]:
loc_df

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False
5,2012-05-12,Holiday,Local,Puyo,Cantonizacion del Puyo,False
...,...,...,...,...,...,...
338,2017-11-12,Holiday,Local,Ambato,Independencia de Ambato,False
339,2017-12-05,Additional,Local,Quito,Fundacion de Quito-1,False
341,2017-12-08,Holiday,Local,Loja,Fundacion de Loja,False
342,2017-12-08,Transfer,Local,Quito,Traslado Fundacion de Quito,False


In [108]:
df=res_df.merge(nat_df,on="date",how="left")

In [109]:
res_df.shape

(3029400, 13)

In [110]:
df=df.merge(reg_df,left_on=['date', 'state'], right_on=['date', 'locale_name'],how="left")

In [111]:
df=df.merge(loc_df,left_on=['date', 'city'], right_on=['date', 'locale_name'],how="left")

In [112]:
df.shape

(3029400, 28)

In [113]:
df['firstday'] = df.description_x.apply(lambda x: 1 if x=='Primer dia del ano' else 0)


In [114]:
df = df.drop(columns=['locale_x', 'locale_name_x', 'description_x', 'transferred_x',
                          'locale_y', 'locale_name_y', 'description_y', 'transferred_y',
                          'locale', 'locale_name', 'description', 'transferred'])

In [115]:
df.loc[~df.type_x.isnull(), 'event_type'] = df.type_x.apply(lambda x: x)
df.loc[~df.type_y.isnull(), 'event_type'] = df.type_y.apply(lambda x: x)
df.loc[~df.type.isnull(), 'event_type'] = df.type.apply(lambda x: x)

In [116]:
df.loc[df.event_type.isnull(), 'event_type'] = df.event_type.apply(lambda x: 'norm')


In [117]:
df = df.drop(columns=['type_x', 'type_y', 'type'])
df['isevent'] = df.event_type.apply(lambda x: 'y' if x!='norm' else 'n')

In [118]:
# Adding Easter
df.loc[df.date.isin(['2017-04-16', '2016-03-27', '2015-04-05', '2014-04-20', '2013-03-31']), 'isevent'] = df.isevent.apply(lambda x: 'y')
df.loc[df.date.isin(['2017-04-16', '2016-03-27', '2015-04-05', '2014-04-20', '2013-03-31']), 'event_type'] = df.event_type.apply(lambda x: 'Holiday')

In [119]:
# Adding closure days
df['isclosed'] = df.groupby(by=['date', 'store_nbr'])['sales'].transform(lambda x: 1 if x.sum()==0 else 0)    
df.loc[(df.date.dt.year==2017) & (df.date.dt.month==8) & (df.date.dt.day>=16) , 'isclosed'] = df.isclosed.apply(lambda x: 0)    
df.loc[df.date.isin(['2017-01-01']), 'isevent'] = df.isevent.apply(lambda x: 'n')

In [120]:
df

,id,date,store_nbr,family,sales,onpromotion,TRAINorTEST,city,state,store,cluster,unique_store,new_store,firstday,event_type,isevent,isclosed
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,1,Quito,Pichincha,D,13,0,0,1,Holiday,y,1
1,1,2013-01-01,1,BABY CARE,0.0,0,1,Quito,Pichincha,D,13,0,0,1,Holiday,y,1
2,2,2013-01-01,1,BEAUTY,0.0,0,1,Quito,Pichincha,D,13,0,0,1,Holiday,y,1
3,3,2013-01-01,1,BEVERAGES,0.0,0,1,Quito,Pichincha,D,13,0,0,1,Holiday,y,1
4,4,2013-01-01,1,BOOKS,0.0,0,1,Quito,Pichincha,D,13,0,0,1,Holiday,y,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3029395,3029395,2017-08-31,9,POULTRY,NaN,1,0,Quito,Pichincha,B,6,0,0,0,norm,n,0
3029396,3029396,2017-08-31,9,PREPARED FOODS,NaN,0,0,Quito,Pichincha,B,6,0,0,0,norm,n,0
3029397,3029397,2017-08-31,9,PRODUCE,NaN,1,0,Quito,Pichincha,B,6,0,0,0,norm,n,0
3029398,3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,NaN,9,0,Quito,Pichincha,B,6,0,0,0,norm,n,0


# Oil

In [121]:
oil

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20
...,...,...
1213,2017-08-25,47.65
1214,2017-08-28,46.40
1215,2017-08-29,46.46
1216,2017-08-30,45.96


In [122]:
# Adding missing values
df_oil=oil.set_index('date').resample("D").mean().interpolate(limit_direction='backward').reset_index()

In [123]:
df_oil

,date,dcoilwtico
0,2013-01-01,93.140000
1,2013-01-02,93.140000
2,2013-01-03,92.970000
3,2013-01-04,93.120000
4,2013-01-05,93.146667
...,...,...
1699,2017-08-27,46.816667
1700,2017-08-28,46.400000
1701,2017-08-29,46.460000
1702,2017-08-30,45.960000


In [124]:
px.scatter(oil,x='date',y="dcoilwtico")

In [125]:
px.scatter(df_oil,x='date',y="dcoilwtico")

In [126]:
oil.isnull().sum()

date           0
dcoilwtico    43
dtype: int64

In [127]:
oil.isnull().sum()

date           0
dcoilwtico    43
dtype: int64

In [128]:
df_oil.isnull().sum()

date          0
dcoilwtico    0
dtype: int64

In [129]:
from datetime import date

f_date = date(2013, 1, 1)
l_date = date(2017, 8, 31)
delta = l_date - f_date

delta

datetime.timedelta(days=1703)

In [130]:
for i in [1, 2, 3, 4, 5, 6, 7, 10, 14, 21, 30, 60, 90]:
        df_oil['lagoil_' + str(i) + '_dcoilwtico'] = df_oil['dcoilwtico'].shift(i)

In [131]:
df_oil['oil_week_avg'] = df_oil['dcoilwtico'].rolling(7).mean()
df_oil['oil_2weeks_avg'] = df_oil['dcoilwtico'].rolling(14).mean()
df_oil['oil_month_avg'] = df_oil['dcoilwtico'].rolling(30).mean()

In [132]:
df_oil.dropna(inplace = True)


In [133]:
df_oil

,date,dcoilwtico,lagoil_1_dcoilwtico,lagoil_2_dcoilwtico,lagoil_3_dcoilwtico,lagoil_4_dcoilwtico,lagoil_5_dcoilwtico,lagoil_6_dcoilwtico,lagoil_7_dcoilwtico,lagoil_10_dcoilwtico,lagoil_14_dcoilwtico,lagoil_21_dcoilwtico,lagoil_30_dcoilwtico,lagoil_60_dcoilwtico,lagoil_90_dcoilwtico,oil_week_avg,oil_2weeks_avg,oil_month_avg
90,2013-04-01,97.100000,97.135000,97.170000,97.205000,97.240000,96.530000,95.990,94.550,93.41,93.710000,92.070000,90.516667,97.650000,93.140000,96.910000,95.171429,93.541111
91,2013-04-02,97.230000,97.100000,97.135000,97.170000,97.205000,97.240000,96.530,95.990,93.79,92.440000,92.440000,90.323333,97.460000,93.140000,97.087143,95.513571,93.771333
92,2013-04-03,95.020000,97.230000,97.100000,97.135000,97.170000,97.205000,97.240,96.530,94.17,93.210000,92.470000,90.130000,97.043333,92.970000,96.871429,95.642857,93.934333
93,2013-04-04,93.260000,95.020000,97.230000,97.100000,97.135000,97.170000,97.205,97.240,94.55,92.460000,93.030000,90.880000,96.626667,93.120000,96.302857,95.700000,94.013667
94,2013-04-05,92.760000,93.260000,95.020000,97.230000,97.100000,97.135000,97.170,97.205,95.99,93.410000,93.490000,90.470000,96.210000,93.146667,95.667857,95.653571,94.090000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1699,2017-08-27,46.816667,47.233333,47.650000,47.240000,48.450000,47.650000,47.390,47.790,47.07,47.996667,49.436667,49.720000,44.740000,49.617500,47.490000,47.573571,48.476000
1700,2017-08-28,46.400000,46.816667,47.233333,47.650000,47.240000,48.450000,47.650,47.390,48.59,47.590000,49.370000,49.883333,44.880000,49.630000,47.348571,47.488571,48.359889
1701,2017-08-29,46.460000,46.400000,46.816667,47.233333,47.650000,47.240000,48.450,47.650,48.19,47.570000,49.070000,50.046667,46.020000,48.290000,47.178571,47.409286,48.240333
1702,2017-08-30,45.960000,46.460000,46.400000,46.816667,47.233333,47.650000,47.240,48.450,47.79,46.800000,49.590000,50.210000,45.838000,48.320000,46.822857,47.349286,48.098667


In [134]:
df = df.merge(df_oil, on=['date'], how='left')


In [135]:
df

,id,date,store_nbr,family,sales,onpromotion,TRAINorTEST,city,state,store,...,lagoil_7_dcoilwtico,lagoil_10_dcoilwtico,lagoil_14_dcoilwtico,lagoil_21_dcoilwtico,lagoil_30_dcoilwtico,lagoil_60_dcoilwtico,lagoil_90_dcoilwtico,oil_week_avg,oil_2weeks_avg,oil_month_avg
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,1,Quito,Pichincha,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2013-01-01,1,BABY CARE,0.0,0,1,Quito,Pichincha,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2013-01-01,1,BEAUTY,0.0,0,1,Quito,Pichincha,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,2013-01-01,1,BEVERAGES,0.0,0,1,Quito,Pichincha,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,2013-01-01,1,BOOKS,0.0,0,1,Quito,Pichincha,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3029395,3029395,2017-08-31,9,POULTRY,NaN,1,0,Quito,Pichincha,B,...,47.24,47.39,47.07,48.54,49.19,45.656,47.68,46.825714,47.362857,48.034333
3029396,3029396,2017-08-31,9,PREPARED FOODS,NaN,0,0,Quito,Pichincha,B,...,47.24,47.39,47.07,48.54,49.19,45.656,47.68,46.825714,47.362857,48.034333
3029397,3029397,2017-08-31,9,PRODUCE,NaN,1,0,Quito,Pichincha,B,...,47.24,47.39,47.07,48.54,49.19,45.656,47.68,46.825714,47.362857,48.034333
3029398,3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,NaN,9,0,Quito,Pichincha,B,...,47.24,47.39,47.07,48.54,49.19,45.656,47.68,46.825714,47.362857,48.034333


# Transactions

In [136]:
transaction.head()

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922


In [137]:
transaction.date=pd.to_datetime(transaction['date'], format='%Y-%m-%d ')


In [138]:
def transactions_func (orig_df):
    
    dff = orig_df.copy()
    
    # Merging orig_transactions and final_df
    dfs = df.merge(dff, on=['date', 'store_nbr'], how='left')
    
    # Filling missing values
    dfs.loc[(dfs.transactions.isnull()) & (dfs.isclosed==1), 'transactions'] = dfs.transactions.apply(lambda x: 0)
    group_df = dfs.groupby(by=['store_nbr', 'date']).transactions.first().reset_index()
    group_df['avg_tra'] = group_df.transactions.rolling(15, min_periods=10).mean()
    group_df['16_tra'] = group_df.transactions.shift(16)
    group_df['21_tra'] = group_df.transactions.shift(21)
    group_df['30_tra'] = group_df.transactions.shift(30)
    group_df['60_tra'] = group_df.transactions.shift(60)
    group_df.drop(columns='transactions', inplace=True)
    dfs = dfs.merge(group_df, on=['date', 'store_nbr'], how='left')
    dfs.loc[(dfs.transactions.isnull()) & (dfs.isclosed==0), 'transactions'] = dfs.avg_tra
    dfs.drop(columns='avg_tra', inplace=True)
    dfs.loc[(df.date.dt.year==2017) & (dfs.date.dt.month==8) & (dfs.date.dt.day>=16) , 'transactions'] = dfs.transactions.apply(lambda x: None)    

    dfs['tot_store_day_onprom'] = dfs.groupby(by=['date', 'store_nbr']).onpromotion.transform(lambda x: x.sum())

    return dfs

In [139]:
final_df = transactions_func(transaction)


In [140]:
final_df

,id,date,store_nbr,family,sales,onpromotion,TRAINorTEST,city,state,store,...,lagoil_90_dcoilwtico,oil_week_avg,oil_2weeks_avg,oil_month_avg,transactions,16_tra,21_tra,30_tra,60_tra,tot_store_day_onprom
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,1,Quito,Pichincha,D,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0
1,1,2013-01-01,1,BABY CARE,0.0,0,1,Quito,Pichincha,D,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0
2,2,2013-01-01,1,BEAUTY,0.0,0,1,Quito,Pichincha,D,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0
3,3,2013-01-01,1,BEVERAGES,0.0,0,1,Quito,Pichincha,D,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0
4,4,2013-01-01,1,BOOKS,0.0,0,1,Quito,Pichincha,D,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3029395,3029395,2017-08-31,9,POULTRY,NaN,1,0,Quito,Pichincha,B,...,47.68,46.825714,47.362857,48.034333,None,2155.0,1923.0,2271.0,2352.0,223
3029396,3029396,2017-08-31,9,PREPARED FOODS,NaN,0,0,Quito,Pichincha,B,...,47.68,46.825714,47.362857,48.034333,None,2155.0,1923.0,2271.0,2352.0,223
3029397,3029397,2017-08-31,9,PRODUCE,NaN,1,0,Quito,Pichincha,B,...,47.68,46.825714,47.362857,48.034333,None,2155.0,1923.0,2271.0,2352.0,223
3029398,3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,NaN,9,0,Quito,Pichincha,B,...,47.68,46.825714,47.362857,48.034333,None,2155.0,1923.0,2271.0,2352.0,223


In [141]:
date = {
    'date_start_train': '2013-01-01',
    'date_end_train': '2017-08-15',
    'date_start_test': '2017-08-16',
    'date_end_test': '2017-08-31',
    'date_start_fore': '2016-06-01'
}

In [142]:
diff_train = (pd.Timestamp(date['date_end_train']) - pd.Timestamp(date['date_start_fore'])).days
diff_test = (pd.Timestamp(date['date_end_test']) - pd.Timestamp(date['date_start_fore'])).days

In [143]:
final = final_df.set_index('date').loc[date['date_start_fore']:,:]


In [144]:
final

,id,store_nbr,family,sales,onpromotion,TRAINorTEST,city,state,store,cluster,...,lagoil_90_dcoilwtico,oil_week_avg,oil_2weeks_avg,oil_month_avg,transactions,16_tra,21_tra,30_tra,60_tra,tot_store_day_onprom
date,,,,,,,,,,,,,,,,,,,,,
2016-06-01,2216808,1,AUTOMOTIVE,3.0,0,1,Quito,Pichincha,D,13,...,34.56,49.174286,48.650000,47.024333,1898.0,1794.0,1828.0,1722.0,1295.0,363
2016-06-01,2216809,1,BABY CARE,0.0,0,1,Quito,Pichincha,D,13,...,34.56,49.174286,48.650000,47.024333,1898.0,1794.0,1828.0,1722.0,1295.0,363
2016-06-01,2216810,1,BEAUTY,4.0,0,1,Quito,Pichincha,D,13,...,34.56,49.174286,48.650000,47.024333,1898.0,1794.0,1828.0,1722.0,1295.0,363
2016-06-01,2216811,1,BEVERAGES,2199.0,37,1,Quito,Pichincha,D,13,...,34.56,49.174286,48.650000,47.024333,1898.0,1794.0,1828.0,1722.0,1295.0,363
2016-06-01,2216812,1,BOOKS,0.0,0,1,Quito,Pichincha,D,13,...,34.56,49.174286,48.650000,47.024333,1898.0,1794.0,1828.0,1722.0,1295.0,363
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-08-31,3029395,9,POULTRY,NaN,1,0,Quito,Pichincha,B,6,...,47.68,46.825714,47.362857,48.034333,None,2155.0,1923.0,2271.0,2352.0,223
2017-08-31,3029396,9,PREPARED FOODS,NaN,0,0,Quito,Pichincha,B,6,...,47.68,46.825714,47.362857,48.034333,None,2155.0,1923.0,2271.0,2352.0,223
2017-08-31,3029397,9,PRODUCE,NaN,1,0,Quito,Pichincha,B,6,...,47.68,46.825714,47.362857,48.034333,None,2155.0,1923.0,2271.0,2352.0,223


# Split function

In [145]:
def my_split_func(df, train_start_date='2013-01-01', train_end_date='2017-08-30',
               val_start_date='2017-09-01', val_end_date='2020-01-01'):
    train_start_date = (pd.to_datetime(train_start_date) - pd.to_datetime('2013-01-01')).days
    train_end_date = (pd.to_datetime(train_end_date) - pd.to_datetime('2013-01-01')).days
    val_start_date = (pd.to_datetime(val_start_date) - pd.to_datetime('2013-01-01')).days
    val_end_date = (pd.to_datetime(val_end_date) - pd.to_datetime('2013-01-01')).days
    
    train = df[(df['days_from_2013'] >= train_start_date) & (df['days_from_2013'] <= train_end_date)]
    val = df[(df['days_from_2013'] >= val_start_date) & (df['days_from_2013'] <= val_end_date)]
    return [train.drop(columns=['sales']), val.drop(columns=['sales']), train['sales'], val['sales']]

In [146]:
final.columns

Index(['id', 'store_nbr', 'family', 'sales', 'onpromotion', 'TRAINorTEST',
       'city', 'state', 'store', 'cluster', 'unique_store', 'new_store',
       'firstday', 'event_type', 'isevent', 'isclosed', 'dcoilwtico',
       'lagoil_1_dcoilwtico', 'lagoil_2_dcoilwtico', 'lagoil_3_dcoilwtico',
       'lagoil_4_dcoilwtico', 'lagoil_5_dcoilwtico', 'lagoil_6_dcoilwtico',
       'lagoil_7_dcoilwtico', 'lagoil_10_dcoilwtico', 'lagoil_14_dcoilwtico',
       'lagoil_21_dcoilwtico', 'lagoil_30_dcoilwtico', 'lagoil_60_dcoilwtico',
       'lagoil_90_dcoilwtico', 'oil_week_avg', 'oil_2weeks_avg',
       'oil_month_avg', 'transactions', '16_tra', '21_tra', '30_tra', '60_tra',
       'tot_store_day_onprom'],
      dtype='object')

In [147]:
final.family.value_counts().size

33

In [148]:
def tags_to_dict():
    tags = {
     'AUTOMOTIVE': [4, 7, 30, 10, 'family'],
     'BABY CARE':  [-8, 2, 25, 5, 'family'],
     'BEAUTY': [-8, 7, 25, 5, 'other'],
     'BEVERAGES': [0, 0, 40, 40, 'food'],
     'BOOKS': [0, 0, 55, 15, 'other'],
     'BREAD/BAKERY': [-3, 0, 30, 30, 'food'],
     'CELEBRATION': [-5, 5, 50, 20, 'family'],
     'CLEANING': [-8, 3, 40, 20, 'food'],
     'DAIRY': [-4, 0, 40, 40, 'food'],
     'DELI': [3, 6, 40, 20, 'food'],
     'EGGS': [-4, -5, 40, 20, 'food'],
     'FROZEN FOODS': [-4, -3, 40, 20, 'food'],
     'GROCERY I': [-4, 3, 40, 20, 'food'],
     'GROCERY II': [-4, 3, 40, 20, 'food'],
     'HARDWARE': [10, 10, 30, 20, 'other'],
     'HOME AND KITCHEN I': [-10, 4, 40, 20, 'family'],
     'HOME AND KITCHEN II': [-10, 4, 40, 20, 'family'],
     'HOME APPLIANCES': [0, 4, 40, 20, 'family'],
     'HOME CARE': [-10, 4, 40, 20, 'family'],
     'LADIESWEAR': [-10, 4, 40, 20, 'other'],
     'LAWN AND GARDEN': [-10, 4, 40, 20, 'family'],
     'LINGERIE': [-10, 4, 40, 2, 'other'],
     'LIQUOR,WINE,BEER': [4, 8, 40, 20, 'food'],
     'MAGAZINES': [-6, -7, 50, 20, 'other'],
     'MEATS': [-4, 5, 40, 20, 'food'],
     'PERSONAL CARE': [-5, 5, 40, 20, 'family'],
     'PET SUPPLIES': [-5, 0, 40, 20, 'family'],
     'PLAYERS AND ELECTRONICS': [5, 5, 25, 10, 'other'],
     'POULTRY': [-7, -4, 40, 20, 'food'],
     'PREPARED FOODS': [0, 6, 30, 10, 'food'],
     'PRODUCE': [0, 0, 40, 40, 'other'],
     'SCHOOL AND OFFICE SUPPLIES': [3, 3, 25, 15, 'family'],
     'SEAFOOD': [-5, 8, 40, 20, 'food']
    }
    
    sex_dict = {}
    luxury_dict = {}
    age_mean_dict = {}
    age_var_dict = {}
    type_dict = {}
    for i in tags.keys():
        sex_dict[i] = tags[i][0]
        luxury_dict[i] = tags[i][1]
        age_mean_dict[i] = tags[i][2]
        age_var_dict[i] = tags[i][3]
        type_dict[i] = tags[i][4]
    return [sex_dict, luxury_dict, age_mean_dict, age_var_dict, type_dict]

In [149]:
def get_oil_dict(oil):
    # estimate price of gaps (market don't work on weekends and holidays)
    price_estim = [-1] * (oil['days_from_2013'][oil.shape[0] - 1] + 1)
    price_estim[0] = 93.14
    for i in range(1, oil.shape[0]):
        price_estim[oil['days_from_2013'][i]] = oil['dcoilwtico'][i]

    for i in range (len(price_estim)):
        if price_estim[i] == -1 or math.isnan(price_estim[i]):
            tj = -1
            for j in range(i + 1, len(price_estim)):
                if price_estim[j] != -1 and (not math.isnan(price_estim[j])):
                    tj = j
                    break

            for j in range(i, tj):
                price_estim[j] = ((tj - j) * price_estim[i - 1] + (j - i) * price_estim[tj]) / (tj - i)

            i = tj

    oil_dict = dict(zip(np.arange(len(price_estim)), price_estim))
    return oil_dict

# last stripes

In [150]:
def add_features (orig_df):
    
    df = orig_df.copy()
        
    # Time features
    df['year'] = df.index.year.astype('int')
    df['quarter'] = df.index.quarter.astype('int')
    df['month'] = df.index.month.astype('int')
    df['day'] = df.index.day.astype('int')
    df['dayofweek'] = df.index.day_of_week.astype('int')
#     df['weekofyear'] = df.index.week.astype('int')
    df['isweekend'] = df.dayofweek.apply(lambda x: 1 if x in (5,6) else 0)
    df['startschool'] = df.month.apply(lambda x: 1 if x in (4,5,8,9) else 0)
    
    df['daysinmonth'] = df.index.days_in_month.astype('int')

    # Dummy features
    df = pd.get_dummies(df, columns=['year'], drop_first=True)
    df = pd.get_dummies(df, columns=['quarter'], drop_first=True)
    df = pd.get_dummies(df, columns=['dayofweek'], drop_first=True)
    df = pd.get_dummies(df, columns=['store'], drop_first=True)
    df = pd.get_dummies(df, columns=['event_type'], drop_first=True)
    df = pd.get_dummies(df, columns=['isevent'], drop_first=True)
    df = pd.get_dummies(df, columns=['state'], drop_first=True)

    # DeterministicProcess
    fourierA = CalendarFourier(freq='A', order=5)
    fourierM = CalendarFourier(freq='M', order=2)
    fourierW = CalendarFourier(freq='W', order=4)

    dp = DeterministicProcess(index=df.index,
                          order=1,
                          seasonal=False,
                          constant=False,
                          additional_terms=[fourierA, fourierM, fourierW],
                          drop=True)
    dp_df = dp.in_sample()
    df = pd.concat([df, dp_df], axis=1)
    
    # Outliers
    df['outliers'] = df.sales.apply(lambda x: 1 if x>30000 else 0)
    
    df.drop(columns=['daysinmonth', 'month', 'city'], inplace=True)
    
    return df

In [151]:
import math

def add_my(df):
    # read
    train = pd.read_csv('../input/store-sales-time-series-forecasting/train.csv')
    oil = pd.read_csv('../input/store-sales-time-series-forecasting/oil.csv')
    trans = pd.read_csv('../input/store-sales-time-series-forecasting/transactions.csv')
    
    # add 'days_from_2013' for easy shifting
    df['days_from_2013'] = (pd.to_datetime(df.index.get_level_values(2)) - pd.to_datetime('2013-01-01')).days
    train['days_from_2013'] = (pd.to_datetime(train['date']) - pd.to_datetime('2013-01-01')).dt.days
    oil['days_from_2013'] = (pd.to_datetime(oil['date']) - pd.to_datetime('2013-01-01')).dt.days
    trans['days_from_2013'] = (pd.to_datetime(trans['date']) - pd.to_datetime('2013-01-01')).dt.days
    
    # groupby features
    gr_day = train.groupby('days_from_2013')['sales'].mean()
    gr_store = train.groupby('store_nbr')['sales'].mean()
    gr_family = train.groupby('family')['sales'].mean()

    days = [16, 18, 20, 21, 25, 28, 30, 35, 42, 60, 90, 120, 180, 365]
    for i in days:
        df['days_' + str(i)] = df['days_from_2013'] - i
        df['days_lagged' + str(i)] = df['days_' + str(i)].map(gr_day).fillna(0)
        df = df.drop(columns=['days_' + str(i)])

    df['store_gb'] = df.index.get_level_values(0).map(gr_store)
    df['family_gb'] = df.index.get_level_values(1).map(gr_family)
    
    oil_dict = get_oil_dict(oil)
# lagged oil
    days = [0, 1, 2, 3, 4, 5, 6, 7, 10, 14, 21, 30, 60, 90, 120, 180, 360]
    for i in days:
        df['days_' + str(i)] = df['days_from_2013'] - i
        df['oil_lagged' + str(i)] = df['days_' + str(i)].map(oil_dict)
        df = df.drop(columns=['days_' + str(i)])
        
    # lagged transactions
    # # fill trans dict
    trans_dict = {}
    for ii in range(trans.shape[0]):
        i = trans.loc[ii]
        trans_dict[tuple([i['store_nbr'], i['days_from_2013']])] = i['transactions']
    
    def transaction_get_value(a, b):
        try:
            return trans_dict[tuple([a, (pd.to_datetime(b) - pd.to_datetime('2013-01-01').dt.days)])]
        except:
            return 0

    days = [16, 18, 20, 21, 25, 28, 30, 35, 42, 60, 90, 120, 180, 365]
    for i in days:
        df['days_' + str(i)] = df['days_from_2013'] - i
        df['oil_lagged' + str(i)] = df['days_' + str(i)].map(oil_dict)
        df['trans_lagged' + str(i)] = [transaction_get_value(*a) for a in tuple(zip(df.index.get_level_values(0),
                                                                        df.index.get_level_values(2)))]
        df = df.drop(columns=['days_' + str(i)])

    sex_dict, luxury_dict, age_mean_dict, age_var_dict, type_dict = tags_to_dict()
    df['tag_sex'] = df.index.get_level_values(1).map(sex_dict)
    df['tag_luxury'] = df.index.get_level_values(1).map(luxury_dict)
    df['tag_age_mean'] = df.index.get_level_values(1).map(age_mean_dict)
    df['tag_age_var'] = df.index.get_level_values(1).map(age_var_dict)
    df['tag_type'] = df.index.get_level_values(1).map(type_dict)
    df = pd.get_dummies(df, columns=['tag_type'])
    
    df['tag_age_min'] = df['tag_age_mean'] - df['tag_age_var']
    df['tag_age_max'] = df['tag_age_mean'] + df['tag_age_var']
    return df

In [152]:
df = add_features(final).loc[:date['date_end_test'],:].reset_index().set_index(['store_nbr', 'family', 'date']).sort_index()

df['16_tra'] = df['16_tra'].fillna(0)
df['21_tra'] = df['21_tra'].fillna(0)
df['30_tra'] = df['30_tra'].fillna(0)
df['60_tra'] = df['60_tra'].fillna(0)


In [153]:
df.columns

Index(['id', 'sales', 'onpromotion', 'TRAINorTEST', 'cluster', 'unique_store',
       'new_store', 'firstday', 'isclosed', 'dcoilwtico',
       'lagoil_1_dcoilwtico', 'lagoil_2_dcoilwtico', 'lagoil_3_dcoilwtico',
       'lagoil_4_dcoilwtico', 'lagoil_5_dcoilwtico', 'lagoil_6_dcoilwtico',
       'lagoil_7_dcoilwtico', 'lagoil_10_dcoilwtico', 'lagoil_14_dcoilwtico',
       'lagoil_21_dcoilwtico', 'lagoil_30_dcoilwtico', 'lagoil_60_dcoilwtico',
       'lagoil_90_dcoilwtico', 'oil_week_avg', 'oil_2weeks_avg',
       'oil_month_avg', 'transactions', '16_tra', '21_tra', '30_tra', '60_tra',
       'tot_store_day_onprom', 'day', 'isweekend', 'startschool', 'year_2017',
       'quarter_2', 'quarter_3', 'quarter_4', 'dayofweek_1', 'dayofweek_2',
       'dayofweek_3', 'dayofweek_4', 'dayofweek_5', 'dayofweek_6', 'store_B',
       'store_C', 'store_D', 'store_E', 'event_type_Black F',
       'event_type_Bridge', 'event_type_Cyber M', 'event_type_Dia de ',
       'event_type_Holiday', 'event_type_T

In [154]:
df

id  sales  onpromotion  TRAINorTEST  \
store_nbr family     date                                                   
1         AUTOMOTIVE 2016-06-01  2216808    3.0            0            1   
                     2016-06-02  2218590    1.0            0            1   
                     2016-06-03  2220372    4.0            0            1   
                     2016-06-04  2222154    9.0            0            1   
                     2016-06-05  2223936    2.0            0            1   
...                                  ...    ...          ...          ...   
54        SEAFOOD    2017-08-27  3022139    NaN            0            0   
                     2017-08-28  3023921    NaN            0            0   
                     2017-08-29  3025703    NaN            0            0   
                     2017-08-30  3027485    NaN            0            0   
                     2017-08-31  3029267    NaN            0            0   

                                 cluster  unique_store  new_store  firstday  \
store_nbr family     date                                                     
1         AUTOMOTIVE 2016-06-01       13             0          0         0   
                     2016-06-02       13             0          0         0   
                     2016-06-03       13             0          0         0   
                     2016-06-04       13             0          0         0   
                     2016-06-05       13             0          0         0   
...                                  ...           ...        ...       ...   
54        SEAFOOD    2017-08-27        3             1          0         0   
                     2017-08-28        3             1          0         0   
                     2017-08-29        3             1          0         0   
                     2017-08-30        3             1          0         0   
                     2017-08-31        3             1          0         0   

                                 isclosed  dcoilwtico  ...  cos(1,freq=M)  \
store_nbr family     date                              ...                  
1         AUTOMOTIVE 2016-06-01         0   49.070000  ...       1.000000   
                     2016-06-02         0   49.140000  ...       0.978148   
                     2016-06-03         0   48.690000  ...       0.913545   
                     2016-06-04         0   49.030000  ...       0.809017   
                     2016-06-05         0   49.370000  ...       0.669131   
...                                   ...         ...  ...            ...   
54        SEAFOOD    2017-08-27         0   46.816667  ...       0.528964   
                     2017-08-28         0   46.400000  ...       0.688967   
                     2017-08-29         0   46.460000  ...       0.820763   
                     2017-08-30         0   45.960000  ...       0.918958   
                     2017-08-31         0   47.260000  ...       0.979530   

                                 sin(2,freq=M)  cos(2,freq=M)  \
store_nbr family     date                                       
1         AUTOMOTIVE 2016-06-01       0.000000       1.000000   
                     2016-06-02       0.406737       0.913545   
                     2016-06-03       0.743145       0.669131   
                     2016-06-04       0.951057       0.309017   
                     2016-06-05       0.994522      -0.104528   
...                                        ...            ...   
54        SEAFOOD    2017-08-27      -0.897805      -0.440394   
                     2017-08-28      -0.998717      -0.050649   
                     2017-08-29      -0.937752       0.347305   
                     2017-08-30      -0.724793       0.688967   
                     2017-08-31      -0.394356       0.918958   

                                 sin(1,freq=W-SUN)  cos(1,freq=W-SUN)  \
store_nbr family     date                                               
1         AUTO

In [155]:
# df = add_my(df)
# display(df)

In [156]:
df.isnull()

id  sales  onpromotion  TRAINorTEST  \
store_nbr family     date                                                 
1         AUTOMOTIVE 2016-06-01  False  False        False        False   
                     2016-06-02  False  False        False        False   
                     2016-06-03  False  False        False        False   
                     2016-06-04  False  False        False        False   
                     2016-06-05  False  False        False        False   
...                                ...    ...          ...          ...   
54        SEAFOOD    2017-08-27  False   True        False        False   
                     2017-08-28  False   True        False        False   
                     2017-08-29  False   True        False        False   
                     2017-08-30  False   True        False        False   
                     2017-08-31  False   True        False        False   

                                 cluster  unique_store  new_store  firstday  \
store_nbr family     date                                                     
1         AUTOMOTIVE 2016-06-01    False         False      False     False   
                     2016-06-02    False         False      False     False   
                     2016-06-03    False         False      False     False   
                     2016-06-04    False         False      False     False   
                     2016-06-05    False         False      False     False   
...                                  ...           ...        ...       ...   
54        SEAFOOD    2017-08-27    False         False      False     False   
                     2017-08-28    False         False      False     False   
                     2017-08-29    False         False      False     False   
                     2017-08-30    False         False      False     False   
                     2017-08-31    False         False      False     False   

                                 isclosed  dcoilwtico  ...  cos(1,freq=M)  \
store_nbr family     date                              ...                  
1         AUTOMOTIVE 2016-06-01     False       False  ...          False   
                     2016-06-02     False       False  ...          False   
                     2016-06-03     False       False  ...          False   
                     2016-06-04     False       False  ...          False   
                     2016-06-05     False       False  ...          False   
...                                   ...         ...  ...            ...   
54        SEAFOOD    2017-08-27     False       False  ...          False   
                     2017-08-28     False       False  ...          False   
                     2017-08-29     False       False  ...          False   
                     2017-08-30     False       False  ...          False   
                     2017-08-31     False       False  ...          False   

                                 sin(2,freq=M)  cos(2,freq=M)  \
store_nbr family     date                                       
1         AUTOMOTIVE 2016-06-01          False          False   
                     2016-06-02          False          False   
                     2016-06-03          False          False   
                     2016-06-04          False          False   
                     2016-06-05          False          False   
...                                        ...            ...   
54        SEAFOOD    2017-08-27          False          False   
                     2017-08-28          False          False   
                     2017-08-29          False          False   
                     2017-08-30          False          False   
                     2017-08-31          False          False   

                                 sin(1,freq=W-SUN)  cos(1,freq=W-SUN)  \
store_nbr family     date                                               
1         AUTOMOTIVE 2016-06-01       